In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#KFold Cross Validation
from sklearn.model_selection import KFold, train_test_split

# ML Libraries
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# Grid Search
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

plt.rcParams['figure.figsize'] = [20, 8]

In [6]:
train=pd.read_csv(r"C:\Users\hp\Downloads\Bigmart_Train(2).csv")
test=pd.read_csv(r"C:\Users\hp\Downloads\Bigmart_Test(2).csv")

In [7]:
train

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Type_Cat,ItemID_Cat,Outlet_Perf,Existence_Yr
0,9.300,Low Fat,0.016047,249.8092,Medium,Tier 1,Supermarket Type1,3735.1380,Perishables,FD,Medium_Perf,14
1,5.920,Regular,0.019278,48.2692,Medium,Tier 3,Supermarket Type2,443.4228,Non_Perishables,DR,Medium_Perf,4
2,17.500,Low Fat,0.016760,141.6180,Medium,Tier 1,Supermarket Type1,2097.2700,Perishables,FD,Medium_Perf,14
3,19.200,Regular,0.022914,182.0950,Small,Tier 3,Grocery Store,732.3800,Perishables,FD,Low_Perf,15
4,8.930,Non Edible,0.013177,53.8614,High,Tier 3,Supermarket Type1,994.7052,Non_Perishables,NC,High_Perf,26
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,214.5218,High,Tier 3,Supermarket Type1,2778.3834,Non_Perishables,FD,High_Perf,26
8519,8.380,Regular,0.046982,108.1570,Medium,Tier 2,Supermarket Type1,549.2850,Non_Perishables,FD,Medium_Perf,11
8520,10.600,Non Edible,0.035186,85.1224,Small,Tier 2,Supermarket Type1,1193.1136,Non_Perishables,NC,Medium_Perf,9
8521,7.210,Regular,0.145221,103.1332,Medium,Tier 3,Supermarket Type2,1845.5976,Non_Perishables,FD,Medium_Perf,4


In [8]:
# Train and Test Set will have n and n-1 features resp
train.shape, test.shape

((8523, 12), (5681, 11))

In [9]:
# Dummy - dummytrain
dummytrain = pd.get_dummies(train, drop_first = True)
dummytest = pd.get_dummies(test, drop_first = True)

# Robust Scaler
from sklearn.preprocessing import RobustScaler
rs = RobustScaler() # Initiate the Machine

dummytrain.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Existence_Yr,Item_Fat_Content_Non Edible,Item_Fat_Content_Regular,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Cat_Perishables,ItemID_Cat_FD,ItemID_Cat_NC,Outlet_Perf_Low_Perf,Outlet_Perf_Medium_Perf
0,9.30,0.016047,249.8092,3735.1380,14,0,0,1,0,0,0,1,0,0,1,1,0,0,1
1,5.92,0.019278,48.2692,443.4228,4,0,1,1,0,0,1,0,1,0,0,0,0,0,1
2,17.50,0.016760,141.6180,2097.2700,14,0,0,1,0,0,0,1,0,0,1,1,0,0,1
3,19.20,0.022914,182.0950,732.3800,15,0,1,0,1,0,1,0,0,0,1,1,0,1,0
4,8.93,0.013177,53.8614,994.7052,26,1,0,0,0,0,1,1,0,0,0,0,1,0,0


In [10]:
data = dummytrain.loc[:, ["Item_Weight", "Item_Visibility", 
                          "Item_MRP", "Existence_Yr"]]

scaled = pd.DataFrame(rs.fit_transform(data), columns = data.columns)

In [11]:
# Put it back in Dummytrain
dummytrain.loc[:, data.columns] = scaled

In [12]:
dummytrain.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Existence_Yr,Item_Fat_Content_Non Edible,Item_Fat_Content_Regular,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Cat_Perishables,ItemID_Cat_FD,ItemID_Cat_NC,Outlet_Perf_Low_Perf,Outlet_Perf_Medium_Perf
0,-0.409175,-0.627833,1.163142,3735.1380,0.000000,0,0,1,0,0,0,1,0,0,1,1,0,0,1
1,-0.828270,-0.579281,-1.031872,443.4228,-0.588235,0,1,1,0,0,1,0,1,0,0,0,0,0,1
2,0.607564,-0.617122,-0.015191,2097.2700,0.000000,0,0,1,0,0,0,1,0,0,1,1,0,0,1
3,0.818351,-0.524638,0.425652,732.3800,0.058824,0,1,0,1,0,1,0,0,0,1,1,0,1,0
4,-0.455053,-0.670967,-0.970966,994.7052,0.705882,1,0,0,0,0,1,1,0,0,0,0,1,0,0


In [13]:
data = dummytest.loc[:, ["Item_Weight", "Item_Visibility", 
                          "Item_MRP", "Existence_Yr"]]
scaled_test = pd.DataFrame(rs.transform(data), columns = data.columns)

# Put it back in Dummytest
dummytest.loc[:, data.columns] = scaled_test

In [14]:
# Simple Model - Train test Split using Sklearn
lr = LinearRegression() # Initiated the Machine

# Split the Data in Train and Validation Set
X = dummytrain.drop("Item_Outlet_Sales", axis = 1)
y = dummytrain.Item_Outlet_Sales

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.20,
                                               random_state = 0)

# Model Building
pred_train = lr.fit(xtrain, ytrain).predict(xtrain) # calc the Train RMSE
pred = lr.fit(xtrain, ytrain).predict(xtest)

# RMSE on train and validation set
from sklearn.metrics import r2_score, mean_squared_error

r2 = r2_score(ytest, pred)
print("R Squared: ", r2)

train_rmse = np.sqrt(mean_squared_error(ytrain, pred_train))
test_rmse = np.sqrt(mean_squared_error(ytest, pred))

print("Train RMSE: ", train_rmse)
print("Test RMSE: ", test_rmse)

R Squared:  0.5629189395979304
Train RMSE:  1127.7622324824522
Test RMSE:  1131.05032306119


In [16]:
dtree = DecisionTreeRegressor()

# Model Building
pred_train = dtree.fit(xtrain, ytrain).predict(xtrain) # calc the Train RMSE
pred = dtree.fit(xtrain, ytrain).predict(xtest)

# RMSE on train and validation set

r2 = r2_score(ytest, pred)
print("R Squared: ", r2)

train_rmse = np.sqrt(mean_squared_error(ytrain, pred_train))
test_rmse = np.sqrt(mean_squared_error(ytest, pred))


print("Train RMSE: ", train_rmse)
print("Test RMSE: ", test_rmse)

R Squared:  0.2385236355030632
Train RMSE:  4.769494046594517e-15
Test RMSE:  1492.8946029593137


In [17]:
import statsmodels.api as sma

x_train = sma.add_constant(xtrain)
x_test = sma.add_constant(xtest)

sma.OLS(ytrain, x_train).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     583.4
Date:                Wed, 04 Jan 2023   Prob (F-statistic):               0.00
Time:                        18:41:26   Log-Likelihood:                -57591.
No. Observations:                6818   AIC:                         1.152e+05
Df Residuals:                    6802   BIC:                         1.153e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                          9.822e+14   1.12e+15      0.880      0.379   -1.21e+15    3.17e+15
Item_Weight                     -12.9819     23.845     -0.544      0.586     -59.725      33.761
Item_Visibility                 -13.1173     19.116     -0.686      0.493     -50.590      24.356
Item_MRP                       1426.1115     20.160     70.741      0.000    1386.592    1465.631
Existence_Yr                   -599.1103    193.498     -3.096      0.002    -978.428    -219.793
Item_Fat_Content_Non Edible    4.124e+14   4.69e+14      0.880      0.379   -5.06e+14    1.33e+15
Item_Fat_Content_Regular         35.0427     31.566      1.110      0.267     -26.837      96.923
Outlet_Size_Medium              7.76e+14   8.82e+14      0.880      0.379   -9.53e+14     2.5e+15
Outlet_Size_Small               7.76e+14   8.82e+14      0.880      0.379   -9.53e+14     2.5e+15
Outlet_Location_Type_Tier 2    -203.8833     79.988     -2.549      0.011    -360.684     -47.082
Outlet_Location_Type_Tier 3    -435.4890    166.801     -2.611      0.009    -762.471    -108.507
Outlet_Type_Supermarket Type1 -9.822e+14   1.12e+15     -0.880      0.379   -3.17e+15    1.21e+15
Outlet_Type_Supermarket Type2 -9.822e+14   1.12e+15     -0.880      0.379   -3.17e+15    1.21e+15
Outlet_Type_Supermarket Type3 -1.758e+15      2e+15     -0.880      0.379   -5.67e+15    2.16e+15
Item_Type_Cat_Perishables       -16.8995     31.442     -0.537      0.591     -78.537      44.738
ItemID_Cat_FD                    30.2888     50.634      0.598      0.550     -68.969     129.547
ItemID_Cat_NC                 -4.124e+14   4.69e+14     -0.880      0.379   -1.33e+15    5.06e+14
Outlet_Perf_Low_Perf          -1.758e+15      2e+15     -0.880      0.379   -5.67e+15    2.16e+15
Outlet_Perf_Medium_Perf        -7.76e+14   8.82e+14     -0.880      0.379    -2.5e+15    9.53e+14
==============================================================================
Omnibus:                      828.566   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2053.073
Skew:                           0.700   Prob(JB):                         0.00
Kurtosis:                       5.295   Cond. No.                     2.35e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.4e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
# Cross Validation Model

kfold = KFold(n_splits=15, shuffle=True, random_state = 0)

pred = []
for train_index, test_index in kfold.split(X, y):
    #print(train_index, test_index)
    xtrain = X.loc[train_index] # input/predictors
    ytrain = y.loc[train_index]
    xtest = X.loc[test_index]
    ytest = y.loc[test_index]
    print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)
    # Build the Model
    pred.append(lr.fit(xtrain, ytrain).predict(xtest))

(7954, 18) (569, 18) (7954,) (569,)
(7954, 18) (569, 18) (7954,) (569,)
(7954, 18) (569, 18) (7954,) (569,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)
(7955, 18) (568, 18) (7955,) (568,)


In [21]:
finalpred = pd.DataFrame(pred).T.mean(axis=1)

In [20]:
finalpred

0      2278.971406
1      1599.347065
2      2007.411355
3      2219.406860
4      2418.959376
          ...     
564    2351.806279
565    2489.143815
566    2184.532150
567    2122.473402
568    1965.257226
Length: 569, dtype: float64

In [23]:
np.sqrt(mean_squared_error(ytest, finalpred[1:]))

1631.2619680045627

In [28]:
dummytest

,Item_Weight,Item_Visibility,Item_MRP,Existence_Yr,Item_Fat_Content_Non Edible,Item_Fat_Content_Regular,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Cat_Perishables,ItemID_Cat_FD,ItemID_Cat_NC,Outlet_Perf_Low_Perf,Outlet_Perf_Medium_Perf
0,1.010539,-0.755300,-0.382832,0.000000,0,0,1,0,0,0,1,0,0,0,1,0,0,1
1,-0.533168,-0.291519,-0.606564,-0.470588,0,1,0,1,1,0,1,0,0,1,1,0,0,1
2,0.247985,0.627352,1.075409,0.058824,1,0,0,1,0,1,0,0,0,0,0,1,1,0
3,-0.655301,-0.637734,0.130925,-0.470588,0,0,0,1,1,0,1,0,0,0,1,0,0,1
4,0.123993,0.913235,0.993465,0.823529,0,1,1,0,0,1,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,-0.260384,-0.666164,-0.018487,0.117647,0,1,0,1,0,0,1,0,0,0,1,0,0,1
5677,-0.619963,1.279772,0.284609,-0.588235,0,1,1,0,0,1,0,1,0,0,1,0,0,1
5678,-0.322381,0.235949,-0.264316,-0.176471,1,0,1,0,1,0,1,0,0,0,0,1,0,1
5679,0.334780,0.405794,0.779908,-0.470588,0,1,0,1,1,0,1,0,0,0,1,0,0,1


In [29]:
dtree = DecisionTreeRegressor()
pred_tree = dtree.fit(X, y).predict(dummytest) #Predictions

In [42]:
pred_lr = lr.fit(X, y).predict(dummytest) #Predictions

In [35]:
sample = pd.read_csv(r"C:\Users\hp\Downloads\sample_submission_8RXa3c6.csv")
sample["Item_Outlet_Sales"] = pred_tree


In [39]:
sample

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1693.7952
1,FDW14,OUT017,959.4178
2,NCN55,OUT010,1775.6886
3,FDQ58,OUT017,612.5360
4,FDY38,OUT027,6705.2718
...,...,...,...
5676,FDB58,OUT046,1438.1280
5677,FDD47,OUT018,673.7896
5678,NCO17,OUT045,579.2460
5679,FDJ26,OUT017,1923.4962


In [40]:
sample.to_csv("Dtree.csv", index = False) #1675.1865385926674.

In [43]:
sample2= pd.read_csv(r"C:\Users\hp\Downloads\sample_submission_8RXa3c6.csv")

In [44]:
sample2

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000
...,...,...,...
5676,FDB58,OUT046,1000
5677,FDD47,OUT018,1000
5678,NCO17,OUT045,1000
5679,FDJ26,OUT017,1000


In [46]:
sample2["Item_Outlet_Sales"] = pred_lr

In [47]:
sample2

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1806.3125
1,FDW14,OUT017,1623.1875
2,NCN55,OUT010,1874.6250
3,FDQ58,OUT017,2626.6875
4,FDY38,OUT027,5194.5625
...,...,...,...
5676,FDB58,OUT046,2339.1875
5677,FDD47,OUT018,2435.5625
5678,NCO17,OUT045,1844.9375
5679,FDJ26,OUT017,3581.4375


In [48]:
sample2.to_csv('lr.csv',index=False)

In [50]:
cd

C:\Users\hp
